In [9]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("lucid-parsec-464412-t8-75d0c2e2a5fa.json", scope)
client = gspread.authorize(creds)

sheet = client.open("Spotify History").sheet1

import csv
with open("Spotify_History.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(sheet.row_values(1))  # Write header
    writer.writerows(sheet.get_all_values()[1:])  # Write data

print("Spotify History Sheet downloaded and saved as 'Spotify_History.csv'.")

Spotify History Sheet downloaded and saved as 'Spotify_History.csv'.


In [10]:

sheet = client.open("Recent Spotify Plays").sheet1

import csv
with open("Recent_Spotify_History.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(sheet.row_values(1)) 
    writer.writerows(sheet.get_all_values()[1:]) 

print("Spotify Historydownloaded'.")

Spotify Historydownloaded'.


In [11]:
import pandas as pd

df = pd.read_csv("Recent_Spotify_History.csv")

if "URI" in df.columns:
    df = df.drop(columns=["URI"])

df.to_csv("Recent_Spotify_History.csv", index=False)

print("URI column removed from 'Recent_Spotify_History.csv'.")

URI column removed from 'Recent_Spotify_History.csv'.


In [13]:
import pandas as pd

history_df = pd.read_csv("Archived_CSV/Spotify_History2014-Aug2025.csv")
recent_df = pd.read_csv("Archived_CSV/Recent_Spotify_History.csv")

combined_df = pd.concat([history_df, recent_df], ignore_index=True)

num_duplicates = combined_df.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")

combined_df = combined_df.drop_duplicates()

print("Recent history appended, duplicates removed, and saved to 'Spotify_History2014-Aug2025.csv'.")

Number of duplicate rows: 171
Recent history appended, duplicates removed, and saved to 'Spotify_History2014-Aug2025.csv'.


In [14]:
combined_df.to_csv("Spotify_History2014-Recent.csv", index=False)

In [16]:
import pandas as pd
import re
from datetime import datetime

df = pd.read_csv('Spotify_History2014-Recent.csv')

print(f"Original DataFrame shape: {df.shape}")
print(f"First few date entries:")
print(df['Date/Time'].head())
print(f"\nLast few date entries:")
print(df['Date/Time'].tail())

def convert_to_iso_format(date_str):
    """
    Convert date string to ISO format (YYYY-MM-DDTHH:MM:SSZ)
    Handles both ISO format (already correct) and human-readable format
    """
    
    date_str = str(date_str).strip('"')
    
    if 'T' in date_str and date_str.endswith('Z'):
        return date_str
    
    if ' at ' in date_str:
        try:
            
            dt = datetime.strptime(date_str, "%B %d, %Y at %I:%M%p")
            
            return dt.strftime("%Y-%m-%dT%H:%M:%SZ")
        except ValueError:
            
            print(f"Could not parse date: {date_str}")
            return date_str
    
    return date_str

print("\nConverting date formats...")
df['Date/Time'] = df['Date/Time'].apply(convert_to_iso_format)

print(f"\nAfter conversion - First few date entries:")
print(df['Date/Time'].head())
print(f"\nAfter conversion - Last few date entries:")
print(df['Date/Time'].tail())

non_iso_entries = df[~(df['Date/Time'].str.contains('T') & df['Date/Time'].str.endswith('Z'))]
print(f"\nNumber of entries that couldn't be converted to ISO format: {len(non_iso_entries)}")
if len(non_iso_entries) > 0:
    print("Examples of unconverted entries:")
    print(non_iso_entries['Date/Time'].head())

df.to_csv('Spotify_History2014-Recent-ISO.csv', index=False)
print(f"\nUpdated CSV file saved with {len(df)} entries")

Original DataFrame shape: (179344, 4)
First few date entries:
0    2014-12-19T10:31:24Z
1    2014-12-19T10:31:26Z
2    2014-12-19T10:31:31Z
3    2014-12-22T19:45:10Z
4    2014-12-23T15:54:33Z
Name: Date/Time, dtype: object

Last few date entries:
179339    September 25, 2025 at 12:53PM
179340    September 25, 2025 at 01:03PM
179341    September 25, 2025 at 01:07PM
179342    September 25, 2025 at 01:11PM
179343    September 25, 2025 at 01:14PM
Name: Date/Time, dtype: object

Converting date formats...

After conversion - First few date entries:
0    2014-12-19T10:31:24Z
1    2014-12-19T10:31:26Z
2    2014-12-19T10:31:31Z
3    2014-12-22T19:45:10Z
4    2014-12-23T15:54:33Z
Name: Date/Time, dtype: object

After conversion - Last few date entries:
179339    2025-09-25T12:53:00Z
179340    2025-09-25T13:03:00Z
179341    2025-09-25T13:07:00Z
179342    2025-09-25T13:11:00Z
179343    2025-09-25T13:14:00Z
Name: Date/Time, dtype: object

Number of entries that couldn't be converted to ISO format: